In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from math import ceil
from scipy.sparse import csr_matrix
sparse_matrix = csr_matrix
import shap
from imblearn.over_sampling import SMOTE

c:\Users\Bruno Bartolomeu\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_years = pd.read_csv(r"C:\Users\Bruno Bartolomeu\Downloads\Dataset for candidates\teste_bruno\air_system_previous_years.csv", sep = ';' )


In [3]:
dataset_year = pd.read_csv(r"C:\Users\Bruno Bartolomeu\Downloads\Dataset for candidates\teste_bruno\air_system_present_year.csv", sep = ';' )

In [4]:
print(dataset_years.head())
print(dataset_year.head())

  class  aa_000 ab_000      ac_000 ad_000 ae_000 af_000 ag_000 ag_001 ag_002  \
0   neg   76698     na  2130706438    280      0      0      0      0      0   
1   neg   33058     na           0     na      0      0      0      0      0   
2   neg   41040     na         228    100      0      0      0      0      0   
3   neg      12      0          70     66      0     10      0      0      0   
4   neg   60874     na        1368    458      0      0      0      0      0   

   ...   ee_002  ee_003  ee_004  ee_005  ee_006  ee_007  ee_008 ee_009 ef_000  \
0  ...  1240520  493384  721044  469792  339156  157956   73224      0      0   
1  ...   421400  178064  293306  245416  133654   81140   97576   1500      0   
2  ...   277378  159812  423992  409564  320746  158022   95128    514      0   
3  ...      240      46      58      44      10       0       0      0      4   
4  ...   622012  229790  405298  347188  286954  311560  433954   1218      0   

  eg_000  
0      0  
1      0  

In [5]:
if 'class' not in dataset_years.columns:
    raise KeyError

In [6]:
dataset_years.replace('na', np.nan, inplace=True)
dataset_year.replace('na', np.nan, inplace=True)

In [7]:
dataset_years = dataset_years.apply(pd.to_numeric, errors='coerce')
dataset_year = dataset_year.apply(pd.to_numeric, errors='coerce')

In [8]:
dataset_years.fillna(dataset_years.mean(), inplace=True)
dataset_year.fillna(dataset_year.mean(), inplace=True)

In [9]:
dataset_years.fillna(method='ffill', inplace=True)
dataset_year.fillna(method='ffill', inplace=True)

C:\Users\Bruno Bartolomeu\AppData\Local\Temp\ipykernel_29760\2791480135.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset_years.fillna(method='ffill', inplace=True)
C:\Users\Bruno Bartolomeu\AppData\Local\Temp\ipykernel_29760\2791480135.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset_year.fillna(method='ffill', inplace=True)


In [10]:
dataset_years = pd.get_dummies(dataset_years, drop_first=True)
dataset_year = pd.get_dummies(dataset_year, drop_first=True)

In [11]:
print("Distribuição das classes após o pré-processamento:")
print(dataset_years['class'].value_counts())

Distribuição das classes após o pré-processamento:
Series([], Name: count, dtype: int64)


In [12]:
X = dataset_years.drop('class', axis=1, errors='ignore')
y = dataset_years['class'].apply(lambda x: 1 if x == 'pos' else 0)

In [13]:
print("Distribuição das classes antes da divisão:")
print(y.value_counts())

Distribuição das classes antes da divisão:
class
0    60000
Name: count, dtype: int64


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
print("Distribuição das classes no treino:")
print(y_train.value_counts())
print("Distribuição das classes no teste:")
print(y_test.value_counts())

Distribuição das classes no treino:
class
0    48000
Name: count, dtype: int64
Distribuição das classes no teste:
class
0    12000
Name: count, dtype: int64


In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [18]:
results = {}
for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    auc = roc_auc_score(y_test, y_pred)
    results[model_name] = {
        "Classification Report": classification_report(y_test, y_pred),
        "AUC-ROC": auc
    }

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

Infelizmente parei por aqui pois não consegui entender o motivo do erro e sem tempo já que o prazo expirou ontem.
Sei que não é o que foi solicitado e por isso, agradeço por ter participado do processo.
